# 8.4: Inspecting `HCLG`

In the previous notebooks we ran `run_compile_graph.sh` which built a large `FST` called `HCLG.fst`.  And since we ran this command for **each** of the three acoustic models we built, you will see an `HCLG.fst` in each of the three "main" subdirectories of `exp`: `monophones`, `triphones`, and `triphones_lda`.

In [8]:
%%bash
ls -lah exp/monophones | grep HCLG

In [5]:
%%bash
ls -lah exp/triphones | grep HCLG

In [7]:
%%bash
ls -lah exp/triphones_lda | grep HCLG

The first thing you should notice is that, perhaps not surprisingly, the `HCLG` is larger for each of the subsequent levels of the acoustic model.

Before we inspect the `HCLG` in more detail, take the time to read [this **excellent** blog post](http://vpanayotov.blogspot.com/2012/06/kaldi-decoding-graph-construction.html) (from one of the main contributors of `kaldi`) that describes in detail the makeup of the `HCLG`.  

You may also want to read **Section 6** of [this tutorial](https://github.com/michaelcapizzi/kaldi/blob/kaldi_instructional/egs/INSTRUCTIONAL/resource_files/resources/wfst_tutorial.pdf) which is a summary of the [original paper](https://github.com/michaelcapizzi/kaldi/blob/kaldi_instructional/egs/INSTRUCTIONAL/resource_files/resources/wfst_paper.pdf) introducing the idea of an `HCLG`.

## Building `HCLG`

The `HCLG` is a single `FST` that represents **all** aspects of our `ASR` pipeline into one graph.  As you read, it is the `composition` of **four** separate `FST`s (Note that they are `composed` in reverse order.

Decoding (when we take a new audio file and predict what was said) comes down to two steps:

   1. determining which `GMM` best matches the incoming frames
   2. looking for the most likely path through the graph

We can use the same packages we used last week to take a closer look at `HCLG` and its composites.

In [12]:
# because of the way `kaldi` installed `openFST` we have to add the path to the python functions here
import sys
sys.path.append("/scratch/kaldi/tools/openfst-1.6.2/lib/python2.7/site-packages")    

from utils.fst_manipulate import fst_manipulate as fstman  # scripts to further manipulate fsts

import pywrapfst as openfst  # the wrapper module
import graphviz as dot       # a wrapper for graphviz, which will allow us to visualize

### `G`

`G` is the `FST` representation of our `language model`.  We looked at this in detail in the last week's notebooks.  It has `word:word` on its edges.  Below is our `language model` built from the animal corpus.

In [18]:
G_animal = openfst.Fst.read("resource_files/fst/animal_fst-2_gram.fst")
G_animal

FstIOError: Read failed: 'resource_files/fst/animal_fst-2_gram.fst'

The "real" `G.fst` can be found in `data/lang` and was built way back in week 3 when we ran `run_prepare_data.sh` to build the `data` directory.

**Note:** These composite `FST`s will be too big to visualize, but we can still gather some information about them.

In [20]:
G = openfst.Fst.read("data/lang_test_tg/G.fst")

We can calculate the number of states by converting the `iterator` in `.states()` to a `<list>` and getting its length.

In [23]:
G_states = len(list(G.states()))
G_states

1178044

We can do a similar thing with the `.arcs()` `iterator` with a small modification.  `.arcs()` takes one argument, a `state`, and so we loop through all `state`s, and then count how many `arc`s that state has.

In [24]:
G_arcs = 0
for s in G.states():
    intermediate_arcs = len(list(G.arcs(s)))
    G_arcs += intermediate_arcs
G_arcs

2659792

We can also get some information about the `arc`s.

**Note:** We'll just look at the first **five** states and then `break` out of our nested `for` loops.

In [52]:
c = 0
for s in G.states():
    for a in G.arcs(s):
        if c < 5:
            print("in: {}\nout: {}\nweight: {}".format(
                a.ilabel,
                a.olabel,
                a.weight
                )
            )
            print("=========")
        else:
            break
        c += 1
    break

in: 2
out: 2
weight: 4.15409374
in: 49
out: 49
weight: 10.8683262
in: 220
out: 220
weight: 10.9160538
in: 227
out: 227
weight: 10.9750328
in: 456
out: 456
weight: 7.23240852


Remember that `openfst` will use `index`es instead of `string`s on the `arc`s, but we can recover the words by looking at the "lookup" file `kaldi` built for this very purpose.

In [53]:
%%bash
cat data/lang/words.txt | grep -E " 2$"       # the -E flag will allow us to use a regex
cat data/lang/words.txt | grep -E " 49$"
cat data/lang/words.txt | grep -E " 220$"
cat data/lang/words.txt | grep -E " 227$"
cat data/lang/words.txt | grep -E " 456$"

A 2
AARON 49
ABE 220
ABEL 227
ABOUT 456


You'll notice that the `ilabel` and `olabel` are the same, confirming that the `arc`s consist of `word:word`.

### `L`
`L` is the `FST` representation of our `lexicon`.  This was built in week 3 when we built the `data` directory (see `3.2: Inspecting data dir`).  

In [42]:
ls -lah data/lang/ | grep L

-rw-r--r-- 1 root root  37M Feb  1 21:00 L.fst
-rw-r--r-- 1 root root  38M Feb  1 21:00 L_disambig.fst


In [43]:
L = openfst.Fst.read("data/lang/L.fst")

In [59]:
c = 0
for s in L.states():
    for a in L.arcs(s):
        if c < 5:
            print("in: {}\nout: {}\nweight: {}".format(
                a.ilabel,
                a.olabel,
                a.weight
                )
            )
            print("=========")
        else:
            break
        c += 1
    if c > 5:
        break

in: 0
out: 0
weight: 0.693147182
in: 1
out: 0
weight: 0.693147182
in: 5
out: 1
weight: 0.693147182
in: 5
out: 1
weight: 0.693147182
in: 33
out: 2
weight: 0.693147182


`L` has a different `arc` structure, consisting of `letter:word`.  And so to understand these `arc`s we need to also access the `phones` "lookup".